# **_Sentiment Analysis_**

### Here is everything you are going to need to run this notebook

* nltk.download('all')
* pip install pyenchant --user
* pip install autocorrect --user
* pip install sklearn --user
---

Here we have some globals to help run this notebook better.
There some files that have the results of the cleaning ready.
They take a lot of time so you should propably run the files 
to see the results.

If you want you can of course dont run the part that takes a lot of time(autocorrect).

In [12]:
use_files=1
use_autocorrect=0


The first thing that we are going to do is load all the files that we need.

The twitters datasets first.

In [13]:
import pandas as pd

dataset_gold = pd.read_csv("../twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt", encoding='utf8',delimiter='\t',header=None);
dataset_train = pd.read_csv("../twitter_data/train2017.tsv", encoding='utf8',delimiter='\t',header=None);
dataset_test = pd.read_csv("../twitter_data/test2017.tsv", encoding='utf8',delimiter='\t',header=None);


And now the lexicons

In [14]:
from read_files import read_lexica
all_lex=[] #list of dictionaries
all_lex.append(read_lexica("../lexica/affin/affin.txt"))
all_lex.append(read_lexica("../lexica/emotweet/valence_tweet.txt"))
all_lex.append(read_lexica("../lexica/generic/generic.txt"))
all_lex.append(read_lexica("../lexica/nrc/val.txt"))
all_lex.append(read_lexica("../lexica/nrctag/val.txt"))
all_lex.append(read_lexica("../lexica/sentric/senticnet5.txt"))


Some libraries that we are going to need below are here

In [15]:
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))  

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

from read_files import read_dict

import enchant
d = enchant.Dict("en_US")
from autocorrect import spell

This is were the cleaning is.If you want to skip this part and use the files set the use_flies variable to 1.Also if you want to use the autocorrect set the use_autocorrect variable to 1.

In [16]:
y2 = dataset_train.iloc[:, 2]
y3 = dataset_train.iloc[:, 3]
train_data=list(map(list, zip(y2,y3))) #list of tweets and values

from fix_tweets import clean_tw
fixed_tweets,hash_tags,name_tags=clean_tw(train_data)

words=read_dict()

import csv        
print("asdas")
if use_files==0:
    i=0
    for tweet in fixed_tweets:
        tw=tweet[0]
        tw=tknzr.tokenize(tw)
        tw1=[]
        for word in tw:
            word=word.lower()
            if word not in nltk_stopwords:
                if (not d.check(word)) and (word not in words):
                    if use_autocorrect==1:
                        wdd=spell(word)
                        if wdd!=word :
                            if word!='!':
                                word=wdd

                            tw1.append(stemmer.stem(word))
                    else:
                        tw1.append(stemmer.stem(word))

                else:
                    tw1.append(stemmer.stem(word))

        tweet[0] = " ".join(tw1)
    
    with open('../resources/twitter_fixed.tsv', 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        for tweet in fixed_tweets:
            tsv_output.writerow([tweet[0]])


asdas


### Bag of words


In [21]:
dataset_fix_train = pd.read_csv('../resources/twitter_fixed.tsv', encoding='utf8',delimiter='\t',header=None);

corpus=[]
for i in range(dataset_fix_train.shape[0]):
    tweet = dataset_fix_train.iloc[i, 0]
    corpus.append(str(tweet))

test_tweets=[]   
for i in range(dataset_test.shape[0]):
    tweet = dataset_test.iloc[i, 3]
    test_tweets.append(tweet) 

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=nltk_stopwords ,max_features=2000)

X = vectorizer.fit_transform(corpus).toarray()
y = dataset_train.iloc[:, 2]

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]


MemoryError: 

This are the results of bag of words

In [18]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score

print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

classification_report
              precision    recall  f1-score   support

    negative       0.30      0.38      0.33      3972
     neutral       0.48      0.32      0.39      5937
    positive       0.22      0.30      0.25      2375

    accuracy                           0.34     12284
   macro avg       0.33      0.33      0.32     12284
weighted avg       0.37      0.34      0.34     12284

______________________________________________________
accuracy_score normalized
0.3369423640507978
_________________________
accuracy_score
4139
_________________________
[0.33466667 0.38692207 0.25156334]
0.32438402606118083
0.3369423640507978
0.3438550374464943


For the bag of words above we used unigrams to get those results.In sentiment analysis this is not the best approach.We will now use bigrams and see how our results change.

In [19]:
vectorizer_bigram = CountVectorizer(stop_words=nltk_stopwords ,max_features=2000,ngram_range=(2, 2))
X = vectorizer_bigram.fit_transform(corpus).toarray()
classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer_bigram.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)

In [20]:
The results this time

SyntaxError: invalid syntax (<ipython-input-20-a03b4ae56639>, line 1)

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print("F1_score with average [None,macro,micro,weighted]")
print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

This time the results seem to be worse.This is not what we expected.This might be because we didnt have enought tweets to train under sets of words.

We will try one last time with trigrams so we can catch cases like "not very good"

In [ ]:
vectorizer_trigram = CountVectorizer(stop_words=nltk_stopwords ,max_features=10,ngram_range=(3, 3))
X = vectorizer_trigram.fit_transform(corpus).toarray()
classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer_trigram.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)

The results for trigrams

In [ ]:
print("classification_report")
print(classification_report(y_test, y_pred))
print("______________________________________________________")

print("accuracy_score normalized")
print(accuracy_score(y_test, y_pred))
print("_________________________")
print("accuracy_score" )
print(accuracy_score(y_test, y_pred, normalize=False))
print("_________________________")

print("F1_score with average [None,macro,micro,weighted]")
print(f1_score(y_test, y_pred,average=None))
print(f1_score(y_test, y_pred,average='macro'))
print(f1_score(y_test, y_pred,average='micro'))
print(f1_score(y_test, y_pred,average='weighted'))

### Tf-Idf 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=nltk_stopwords, strip_accents="ascii", lowercase=True)

X = vectorizer.fit_transform(corpus).toarray()

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)

X_test= vectorizer.fit_transform(test_tweets).toarray()
y_pred = classifier.predict(X_test)
y_test=dataset_gold.iloc[:, 1]



from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", strip_accents="ascii", lowercase=True)
#TODO use rafa's tokenization instead of file?
file = open("/home/nem/PycharmProjects/SentimentAnalysis/twitter_data/test.tsv")
tfidf_matrix = vectorizer.fit_transform(file)
file.close()

return tfidf_matrix #.toarray()

### Word Embeddings

In [ ]:
from Word2Vec import WordEmbeddingsVectorize

WordEmbeddingsVectorize(fixed_tweets,size=300,pkl_filename"../resources/word_embed_pickle.pkl")
